<a href="https://colab.research.google.com/github/malipalema/lafand-mt/blob/main/masakhane_lafand.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Evaluating the Masakhane MT models on the English to Isizulu LAFAND test set

## Masakhane - Machine Translation for African Languages (Using JoeyNMT)

In [56]:
! git clone https://github.com/joeynmt/joeynmt.git
! cd joeynmt; pip3 install .

fatal: destination path 'joeynmt' already exists and is not an empty directory.
Processing /content/joeynmt
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
  Created wheel for joeynmt: filename=joeynmt-1.3-py3-none-any.whl size=86029 sha256=37a92db4799add8543292fd34b0b522fad9db3b9ea6ffc5418b9939386169927
  Stored in directory: /tmp/pip-ephem-wheel-cache-9fg36_5_/wheels/0a/f4/bf/6c9d3b8efbfece6cd209f865be37382b02e7c3584df2e28ca4
Successfully built joeynmt
  Attempting uninstall: joeynmt
    Found existing installation: joeynmt 1.3
    Uninstalling joeynmt-1.3:
      Successfully uninstalled joeynmt-1.3


In [57]:
import torch
import pandas as pd
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

device_num = torch.cuda.current_device()
torch.cuda.get_device_name(device_num)
# torch.cuda.is_available()

'Tesla K80'

In [58]:
# Install opus-tools
! pip install opustools-pkg

In [59]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [60]:
import os
import numpy as np
import pandas as pd

source_language = "en"
target_language = "zu" 
lc = True  # If True, lowercase the data.
seed = 42  # Random seed for shuffling.
tag = "baseline" # Give a unique name to your folder - this is to ensure you don't rewrite any models you've already submitted
vocab_size=4000
corpus = "JW300"

os.environ["src"] = source_language # Sets them in bash as well, since we often use bash scripts
os.environ["tgt"] = target_language
os.environ["tag"] = tag
os.environ["vocab_size"] = str(vocab_size)
os.environ["corpus"] = corpus

In [61]:
# This will save it to a folder in our gdrive instead! 
!mkdir -p "/content/drive/My Drive/masakhane/baseline/$src-$trg-$tag"
gdrive_path = f"/content/drive/My Drive/masakhane/baseline/{source_language}-{target_language}-{tag}/"
os.environ["gdrive_path"] = gdrive_path
! echo $gdrive_path

/content/drive/My Drive/masakhane/baseline/en-zu-baseline/


In [62]:
!echo $gdrive_path

/content/drive/My Drive/masakhane/baseline/en-zu-baseline/


In [63]:
# Download the global test set.
! wget https://raw.githubusercontent.com/masakhane-io/masakhane/master/jw300_utils/test/test.en-any.en
  
# And the specific test set for this language pair.
os.environ["trg"] = target_language 
os.environ["src"] = source_language 

! wget https://raw.githubusercontent.com/masakhane-io/masakhane/master/jw300_utils/test/test.en-$trg.en 
! mv test.en-$trg.en test.en
! wget https://raw.githubusercontent.com/masakhane-io/masakhane/master/jw300_utils/test/test.en-$trg.$trg 
! mv test.en-$trg.$trg test.$trg

--2021-11-27 06:34:55--  https://raw.githubusercontent.com/masakhane-io/masakhane/master/jw300_utils/test/test.en-any.en
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 277791 (271K) [text/plain]
Saving to: ‘test.en-any.en.3’

test.en-any.en.3    100%[===================>] 271.28K  --.-KB/s    in 0.04s   

2021-11-27 06:34:56 (7.06 MB/s) - ‘test.en-any.en.3’ saved [277791/277791]

--2021-11-27 06:34:56--  https://raw.githubusercontent.com/masakhane-io/masakhane/master/jw300_utils/test/test.en-zu.en
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response...

In [64]:
# Read the test data to filter from train and dev splits.
# Store english portion in set for quick filtering checks.
en_test_sents = set()
filter_test_sents = "test.en-any.en"
j = 0
blanks = [] # sometimes blank lines creep innto test set - store which lines these are
with open(filter_test_sents) as f:
  for line in f:
    en_test_sents.add(line.strip())
    if len(line)<=1:
      blanks.append(j)
    j += 1
print('Loaded {} global test sentences to filter from the training/dev data.'.format(j))
print(f'There are {len(blanks)} blank lines in the test set')

Loaded 3571 global test sentences to filter from the training/dev data.
There are 0 blank lines in the test set


In [65]:
# filter test set

source_file = f"test.{source_language}"
target_file = f"test.{target_language}"

source = []
target = []

with open(source_file) as f:
  source = f.readlines()
            
with open(target_file) as f:
  target = f.readlines()

df = pd.DataFrame(zip(source, target), columns=['source_sentence', 'target_sentence'])

# remove trailing newline chars
df['source_sentence'] = df['source_sentence'].str.rstrip('" \n')
df['target_sentence'] = df['target_sentence'].str.rstrip('" \n')

# remove leading newline chars
df['source_sentence'] = df['source_sentence'].str.lstrip('"')
df['target_sentence'] = df['target_sentence'].str.lstrip('"')

# remove rows with really short sentences
df = df[~(df['source_sentence'].str.len() <8)] # remove rows wher esource text len <8 characters
df = df[~(df['target_sentence'].str.len() <8)] # remove rows wher esource text len <8 characters

# save the filtered test set
df['source_sentence'].to_csv(f'{source_file}', index=False, header=False, doublequote=False)
df['target_sentence'].to_csv(f'{target_file}', index=False, header=False, doublequote=False)

In [66]:
df.head()

,source_sentence,target_sentence
0,"I am going to make a helper for him , as a com...","Ngizakumenzela umsizi [ “ umphelelisi , ” NW ]..."
1,"Moreover , do not call anyone your father on e...","Ngaphezu kwalokho , ningabizi muntu emhlabeni ..."
2,But the greatest one among you must be your mi...,Kodwa omkhulu kakhulu phakathi kwenu kumelwe a...
3,Jesus said : “ You must love your neighbor as ...,UJesu wathi : “ Kumelwe uthande umakhelwane wa...
4,Some names in this article have been changed .,Amanye amagama kulesi sihloko aye ashintshwa .


In [67]:
# en_jw = read_file(data_dir + 'jw300.en')
# zu_jw = read_file(data_dir + 'jw300.zu')

menzu_train = pd.read_csv('train.tsv', sep='\t')
en_menzu_train = menzu_train['en'].values
zu_menzu_train = menzu_train['zu'].values

menzu_dev = pd.read_csv('dev.tsv', sep='\t')
en_menzu_dev = menzu_dev['en'].values
zu_menzu_dev = menzu_dev['zu'].values

menzu_test = pd.read_csv('test.tsv', sep='\t')
en_menzu_test = menzu_test['en'].values
zu_menzu_test = menzu_test['zu'].values

# merge data
# Train data
train_data_en = list(en_menzu_train)  # en_jw + list(en_menzu_train)
train_data_zu = list(zu_menzu_train)  # zu_jw + list(zu_menzu_train)

df_train_enzu = pd.DataFrame(train_data_en, columns=['source_sentence'])
df_train_enzu['target_sentence'] = train_data_zu

df_train_zuen = pd.DataFrame(train_data_zu, columns=['source_sentence'])
df_train_zuen['target_sentence'] = train_data_en

# dev data
df_dev_enzu = pd.DataFrame(en_menzu_dev, columns=['source_sentence'])
df_dev_enzu['target_sentence'] = zu_menzu_dev

df_dev_zuen = pd.DataFrame(zu_menzu_dev, columns=['source_sentence'])
df_dev_zuen['target_sentence'] = en_menzu_dev

# test data
df_test_enzu = pd.DataFrame(en_menzu_test, columns=['source_sentence'])
df_test_enzu['target_sentence'] = zu_menzu_test

df_test_zuen = pd.DataFrame(zu_menzu_test, columns=['source_sentence'])
df_test_zuen['target_sentence'] = en_menzu_test

In [68]:
df_test_enzu.shape

(998, 2)

In [69]:
# How many samples
size = len(df_test_enzu)
print(f"\n {size} samples in original text")


 998 samples in original text


## Pre-processing and export

It is generally a good idea to remove duplicate translations and conflicting translations from the corpus. In practice, these public corpora include some number of these that need to be cleaned.

In addition we will split our data into dev/test/train and export to the filesystem.

In [70]:
## Preprocessing - Step 1 : Drop NaNs

df_pp = df_test_enzu.dropna()
#df_pp.info(memory_usage='deep')
new_size = len(df_pp)
print(f"\n {size-new_size}({100*(size-new_size)/size :.2f} %) samples removed by dropping all NaNs")
size = new_size


 0(0.00 %) samples removed by dropping all NaNs


In [71]:
## Preprocessing - Step 2a : Drop all duplicates in Source (en) text

df_pp = df_pp.drop_duplicates(subset='source_sentence')
#df_pp.info(memory_usage='deep')
new_size = len(df_pp)
print(f"\n {size-new_size}({100*(size-new_size)/size :.2f} %) samples removed by dropping Source sentence duplicates")
size = new_size


 0(0.00 %) samples removed by dropping Source sentence duplicates


In [72]:
## Preprocessing - Step 2b : Drop all duplicates in Target (zu) text

df_pp = df_pp.drop_duplicates(subset='target_sentence')
#df_pp.info(memory_usage='deep')
new_size = len(df_pp)
print(f"\n {size-new_size}({100*(size-new_size)/size :.2f} %) samples removed by dropping Target sentence duplicates")
size = new_size


 0(0.00 %) samples removed by dropping Target sentence duplicates


In [73]:
##  Preprocessing - Step 3 : Remove all numeric entries

pattern = r"([0-9]*\.?[0-9]*)"  # catch integers and decimals
import re
r = re.compile(pattern)

In [74]:
%%time
##  Preprocessing - Step 3a : Remove all numeric entries - Source text

df_pp['source_sentence'] = df_pp['source_sentence'].str.replace(pattern,"")
df_pp['source_sentence'] = df_pp['source_sentence'].replace("",np.nan)

df_pp = df_pp.dropna()
#df_pp.info(memory_usage='deep')
new_size = len(df_pp)

print(f"\n {size-new_size}({100*(size-new_size)/size :.2f} %) samples removed by dropping nummeric entries from source text")
size = new_size


 0(0.00 %) samples removed by dropping nummeric entries from source text
CPU times: user 66.9 ms, sys: 5.16 ms, total: 72.1 ms
Wall time: 74.2 ms


In [75]:
# DOnt RUN
# Install fuzzy wuzzy to remove "almost duplicate" sentences in the
# test and training sets.
! pip install fuzzywuzzy
! pip install python-Levenshtein
import time
from fuzzywuzzy import process
import numpy as np
from os import cpu_count
from functools import partial
from multiprocessing import Pool


# reset the index of the training set after previous filtering
df_pp.reset_index(drop=False, inplace=True)

# Remove samples from the training data set if they "almost overlap" with the
# samples in the test set.

# Filtering function. Adjust pad to narrow down the candidate matches to
# within a certain length of characters of the given sample.
def fuzzfilter(sample, candidates, pad):
  candidates = [x for x in candidates if len(x) <= len(sample)+pad and len(x) >= len(sample)-pad] 
  if len(candidates) > 0:
    return process.extractOne(sample, candidates)[1]
  else:
    return np.nan

In [76]:
# Dont RUN
start_time = time.time()
### iterating over pandas dataframe rows is not recomended, let use multi processing to apply the function

with Pool(cpu_count()-1) as pool:
    scores = pool.map(partial(fuzzfilter, candidates=list(en_test_sents), pad=5), df_pp['source_sentence'])
hours, rem = divmod(time.time() - start_time, 3600)
minutes, seconds = divmod(rem, 60)
print("done in {}h:{}min:{}seconds".format(hours, minutes, seconds))

# Filter out "almost overlapping samples"
df_pp = df_pp.assign(scores=scores)
df_pp = df_pp[df_pp['scores'] < 95]

done in 0.0h:0.0min:18.56296443939209seconds


In [77]:
# Dont Run
# This section does the split between train/dev for the parallel corpora then saves them as separate files
# We use 1000 dev test and the given test set.
import csv

# TODO: if your corpus is smaller than 1000, reduce this number. With a corpus that small you might not obtain good results with NMT though :/
# Do the split between dev/train and create parallel corpora
num_dev_patterns = 100

# Optional: lower case the corpora - this will make it easier to generalize, but without proper casing.
if lc:  # Julia: making lowercasing optional
    df_pp["source_sentence"] = df_pp["source_sentence"].str.lower()
    df_pp["target_sentence"] = df_pp["target_sentence"].str.lower()

# Julia: test sets are already generated
dev = df_pp.tail(num_dev_patterns) # Herman: Error in original
stripped = df_pp.drop(df_pp.tail(num_dev_patterns).index)

with open("train."+source_language, "w") as src_file, open("train."+target_language, "w") as trg_file:
  for index, row in stripped.iterrows():
    src_file.write(row["source_sentence"]+"\n")
    trg_file.write(row["target_sentence"]+"\n")
    
with open("dev."+source_language, "w") as src_file, open("dev."+target_language, "w") as trg_file:
  for index, row in dev.iterrows():
    src_file.write(row["source_sentence"]+"\n")
    trg_file.write(row["target_sentence"]+"\n")

#stripped[["source_sentence"]].to_csv("train."+source_language, header=False, index=False)  # Herman: Added `header=False` everywhere
#stripped[["target_sentence"]].to_csv("train."+target_language, header=False, index=aFalse)  # Julia: Problematic handling of quotation marks.

#dev[["source_sentence"]].to_csv("dev."+source_language, header=False, index=False)
#dev[["target_sentence"]].to_csv("dev."+target_language, header=False, index=False)


# TODO: Doublecheck the format below. There should be no extra quotation marks or weird characters. It should also not be empty.
! head train.*
! head dev.*

35

37

264

379

44

...

71

92

81

88

==> train.bpe.en <==
peter v@@ an sant@@ : and it means wh@@ at@@ ?
the cost to society will be subst@@ ant@@ i@@ al, the report says in al@@ one, it est@@ im@@ ates a $ billion bur@@ den from health car@@ e, long-@@ term case and hos@@ p@@ ice com@@ b@@ in@@ ed medic@@ are and medic@@ a@@ id will c@@ over $ billion of that, with out@@ -@@ of@@ -@@ p@@ oc@@ ket cost@@ s to car@@ eg@@ iv@@ ers re@@ ach@@ ing $ billion
it is now up to them to make the most of it
"the c@@ ps is car@@ ef@@ ul@@ ly con@@ sid@@ ering all the av@@ ail@@ able inform@@ ation, including the imp@@ act on harr@@ y's famil@@ y, in order to make an independent and ob@@ ject@@ ive char@@ g@@ ing decision
tv aud@@ ienc@@ es were left ou@@ trag@@ ed after a te@@ en@@ age g@@ ir@@ l appe@@ ared on ch@@ an@@ ne@@ l 's hours in a@@ &@@ e compl@@ aining of a bro@@ k@@ en f@@ ing@@ er na@@ il
the dis@@ gu@@ st@@ ing no@@ te re@@ ad@@ : '@@ p@@ ut your do@@ g on a lead@@ , sl@@ ag@@ !
sen sen ce@@ o sub@@ h@@ ash chall@@ a
s

In [78]:
# Install JoeyNMT
! git clone https://github.com/joeynmt/joeynmt.git
! cd joeynmt; pip3 install .
# Install Pytorch with GPU support v1.7.1.
#! pip install torch==1.9.0+cu101 -f https://download.pytorch.org/whl/torch_stable.html

fatal: destination path 'joeynmt' already exists and is not an empty directory.
Processing /content/joeynmt
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
  Created wheel for joeynmt: filename=joeynmt-1.3-py3-none-any.whl size=86029 sha256=dc6b8c61016c0ae636b3d7e3bab46fc4ba367f11f6d66e4ee95b96253e38fb85
  Stored in directory: /tmp/pip-ephem-wheel-cache-5ff9mvi3/wheels/0a/f4/bf/6c9d3b8efbfece6cd209f865be37382b02e7c3584df2e28ca4
Successfully built joeynmt
  Attempting uninstall: joeynmt
    Found existing installation: joeynmt 1.3
    Uninstalling joeynmt-1.3:
      Successfully uninstalled joeynmt-1.3


In [79]:
# One of the huge boosts in NMT performance was to use a different method of tokenizing. 
# Usually, NMT would tokenize by words. However, using a method called BPE gave amazing boosts to performance

# Do subword NMT
from os import path
os.environ["src"] = source_language # Sets them in bash as well, since we often use bash scripts
os.environ["tgt"] = target_language

# Learn BPEs on the training data.
os.environ["data_path"] = path.join("joeynmt", "data", source_language + target_language) # Herman! 
! subword-nmt learn-joint-bpe-and-vocab --input train.$src train.$tgt -s 4000 -o bpe.codes.4000 --write-vocabulary vocab.$src vocab.$tgt

# Apply BPE splits to the development and test data.
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$src < train.$src > train.bpe.$src
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$tgt < train.$tgt > train.bpe.$tgt

! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$src < dev.$src > dev.bpe.$src
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$tgt < dev.$tgt > dev.bpe.$tgt
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$src < test.$src > test.bpe.$src
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$tgt < test.$tgt > test.bpe.$tgt

# Create directory, move everyone we care about to the correct location
! mkdir -p "$data_path"
! cp train.* "$data_path"
! cp test.* "$data_path"
! cp dev.* "$data_path"
! cp bpe.codes.4000 "$data_path"
! ls "$data_path"

# Also move everything we care about to a mounted location in google drive (relevant if running in colab) at gdrive_path
! cp train.* "$gdrive_path"
! cp test.* "$gdrive_path"
! cp dev.* "$gdrive_path"
! cp bpe.codes.4000 "$gdrive_path"
! ls "$gdrive_path"

# Create that vocab using build_vocab
! sudo chmod 777 joeynmt/scripts/build_vocab.py
! joeynmt/scripts/build_vocab.py joeynmt/data/$src$tgt/train.bpe.$src joeynmt/data/$src$tgt/train.bpe.$tgt --output_path joeynmt/data/$src$tgt/vocab.txt

# Some output
! echo "BPE Test language Sentences"
! tail -n 5 test.bpe.$tgt
! echo "Combined BPE Vocab"
! tail -n 10 joeynmt/data/$src$tgt/vocab.txt  # Herman

bpe.codes.4000	dev.zu		test.en-any.en.1  train.bpe.en	vocab.txt
dev.bpe.en	test.bpe.en	test.en-any.en.2  train.bpe.zu
dev.bpe.zu	test.bpe.zu	test.en-any.en.3  train.en
dev.en		test.en		test.tsv	  train.tsv
dev.tsv		test.en-any.en	test.zu		  train.zu
bpe.codes.4000	dev.tsv      test.en	       test.en-any.en.3  train.bpe.zu
dev.bpe.en	dev.zu	     test.en-any.en    test.tsv		 train.en
dev.bpe.zu	test.bpe.en  test.en-any.en.1  test.zu		 train.tsv
dev.en		test.bpe.zu  test.en-any.en.2  train.bpe.en	 train.zu
BPE Test language Sentences
"@@ N@@ g@@ en@@ xa y@@ alokho , ngang@@ aziwa njengom@@ untu ong@@ ath@@ emb@@ ekile ."
"@@ L@@ apho ng@@ if@@ und@@ a iq@@ in@@ iso , ngen@@ qaba ukuqhubeka nal@@ ow@@ o m@@ kh@@ uba , na@@ kuba lo m@@ sebenzi waw@@ ung@@ i@@ hol@@ ela kahle kakhulu ."
N@@ g@@ iy@@ is@@ ibon@@ elo es@@ ihl@@ e em@@ ad@@ od@@ an@@ eni ami amab@@ ili futhi seng@@ iye ng@@ af@@ anel@@ ekela amal@@ ungelo eb@@ andl@@ eni .
"@@ K@@ ub@@ ac@@ wan@@ ingi - m@@ abh@@ uku ent@@ ela 

In [80]:
# Also move everything we care about to a mounted location in google drive (relevant if running in colab) at gdrive_path
! cp train.* "$gdrive_path"
! cp test.* "$gdrive_path"
! cp dev.* "$gdrive_path"
! cp bpe.codes.4000 "$gdrive_path"
! ls "$gdrive_path"

bpe.codes.4000	dev.tsv      test.en	       test.en-any.en.3  train.bpe.zu
dev.bpe.en	dev.zu	     test.en-any.en    test.tsv		 train.en
dev.bpe.zu	test.bpe.en  test.en-any.en.1  test.zu		 train.tsv
dev.en		test.bpe.zu  test.en-any.en.2  train.bpe.en	 train.zu


In [81]:
# This creates the config file for our JoeyNMT system. It might seem overwhelming so we've provided a couple of useful parameters you'll need to update
# (You can of course play with all the parameters if you'd like!)

name = '%s%s' % (source_language, target_language)
gdrive_path = os.environ["gdrive_path"]

# Create the config
config = """
name: "{name}_transformer"

data:
    src: "{source_language}"
    trg: "{target_language}"
    train: "data/{name}/train.bpe"
    dev:   "data/{name}/dev.bpe"
    test:  "data/{name}/test.bpe"
    level: "bpe"
    lowercase: False
    max_sent_length: 100
    src_vocab: "data/{name}/vocab.txt"
    trg_vocab: "data/{name}/vocab.txt"

testing:
    beam_size: 5
    alpha: 1.0
    sacrebleu:                      # sacrebleu options
        remove_whitespace: True     # `remove_whitespace` option in sacrebleu.corpus_chrf() function (defalut: True)
        tokenize: "none"            # `tokenize` option in sacrebleu.corpus_bleu() function (options include: "none" (use for already tokenized test data), "13a" (default minimal tokenizer), "intl" which mostly does punctuation and unicode, etc) 

training:
    #load_model: "{gdrive_path}/models/{name}_transformer/1.ckpt" # if uncommented, load a pre-trained model from this checkpoint
    random_seed: 25
    optimizer: "adam"
    normalization: "tokens"
    adam_betas: [0.9, 0.999] 
    scheduling: "plateau"           # TODO: try switching from plateau to Noam scheduling
    patience: 5                     # For plateau: decrease learning rate by decrease_factor if validation score has not improved for this many validation rounds.
    learning_rate_factor: 0.5       # factor for Noam scheduler (used with Transformer)
    learning_rate_warmup: 1000      # warmup steps for Noam scheduler (used with Transformer)
    decrease_factor: 0.7
    loss: "crossentropy"
    learning_rate: 0.0003
    learning_rate_min: 0.00000001
    weight_decay: 0.0
    label_smoothing: 0.1
    batch_size: 4096
    batch_type: "token"
    eval_batch_size: 3600
    eval_batch_type: "token"
    batch_multiplier: 1
    early_stopping_metric: "ppl"
    epochs: 30                     # TODO: Decrease for when playing around and checking of working. Around 30 is sufficient to check if its working at all
    validation_freq: 100          # TODO: Set to at least once per epoch.
    logging_freq: 100
    eval_metric: "bleu"
    model_dir: "models/{name}_transformer"
    overwrite: True               # TODO: Set to True if you want to overwrite possibly existing models. 
    shuffle: True
    use_cuda: True
    max_output_length: 100
    print_valid_sents: [0, 1, 2, 3]
    keep_last_ckpts: 1

model:
    initializer: "xavier"
    bias_initializer: "zeros"
    init_gain: 1.0
    embed_initializer: "xavier"
    embed_init_gain: 1.0
    tied_embeddings: True
    tied_softmax: True
    encoder:
        type: "transformer"
        num_layers: 6
        num_heads: 4             # TODO: Increase to 8 for larger data.
        embeddings:
            embedding_dim: 256   # TODO: Increase to 512 for larger data.
            scale: True
            dropout: 0.2
        # typically ff_size = 4 x hidden_size
        hidden_size: 256         # TODO: Increase to 512 for larger data.
        ff_size: 1024            # TODO: Increase to 2048 for larger data.
        dropout: 0.3
    decoder:
        type: "transformer"
        num_layers: 6
        num_heads: 4              # TODO: Increase to 8 for larger data.
        embeddings:
            embedding_dim: 256    # TODO: Increase to 512 for larger data.
            scale: True
            dropout: 0.2
        # typically ff_size = 4 x hidden_size
        hidden_size: 256         # TODO: Increase to 512 for larger data.
        ff_size: 1024            # TODO: Increase to 2048 for larger data.
        dropout: 0.3
""".format(name=name, gdrive_path=os.environ["gdrive_path"], source_language=source_language, target_language=target_language)
with open("joeynmt/configs/transformer_{name}.yaml".format(name=name),'w') as f:
    f.write(config)

3514

In [86]:
# Train the model
# You can press Ctrl-C to stop. And then run the next cell to save your checkpoints! 
!cd joeynmt; python3 -m joeynmt train configs/transformer_$src$tgt.yaml

2021-11-27 06:36:54,311 - INFO - root - Hello! This is Joey-NMT (version 1.3).
2021-11-27 06:36:54,320 - INFO - joeynmt.data - Loading training data...
2021-11-27 06:36:54,331 - INFO - joeynmt.data - Building vocabulary...
2021-11-27 06:36:54,588 - INFO - joeynmt.data - Loading dev data...
2021-11-27 06:36:54,590 - INFO - joeynmt.data - Loading test data...
2021-11-27 06:36:54,621 - INFO - joeynmt.data - Data loaded.
2021-11-27 06:36:54,621 - INFO - joeynmt.model - Building an encoder-decoder model...
2021-11-27 06:36:54,924 - INFO - joeynmt.model - Enc-dec model built.
2021-11-27 06:36:57,240 - INFO - joeynmt.training - Total params: 12085760
2021-11-27 06:36:57,244 - WARNING - joeynmt.training - `keep_last_ckpts` option is outdated. Please use `keep_best_ckpts`, instead.
2021-11-27 06:37:00,077 - INFO - joeynmt.helpers - cfg.name                           : enzu_transformer
2021-11-27 06:37:00,077 - INFO - joeynmt.helpers - cfg.data.src                       : en
2021-11-27 06:37:00,

In [90]:
# Copy the created models from the notebook storage to google drive for persistent storage 
! mkdir -p "$gdrive_path/models/${src}${tgt}_transformer/"
! cp -r joeynmt/models/${src}${tgt}_transformer/* "$gdrive_path/models/${src}${tgt}_transformer/"

In [91]:
# Output our validation accuracy
! cat "$gdrive_path/models/${src}${tgt}_transformer/validations.txt"

Steps: 100	Loss: 26223.75195	PPL: 446.47556	bleu: 0.00000	LR: 0.00030000	*
Steps: 200	Loss: 26019.88477	PPL: 425.79233	bleu: 0.00000	LR: 0.00030000	*
Steps: 300	Loss: 25975.14453	PPL: 421.38290	bleu: 0.00000	LR: 0.00030000	*
Steps: 400	Loss: 25336.85938	PPL: 363.22946	bleu: 0.02892	LR: 0.00030000	*
Steps: 500	Loss: 24907.39844	PPL: 328.68948	bleu: 0.04200	LR: 0.00030000	*


In [92]:
# Test our model
! cd joeynmt; python3 -m joeynmt test "$gdrive_path/models/${src}${tgt}_transformer/config.yaml"

2021-11-27 07:16:08,456 - INFO - root - Hello! This is Joey-NMT (version 1.3).
2021-11-27 07:16:08,456 - INFO - joeynmt.data - Building vocabulary...
2021-11-27 07:16:08,702 - INFO - joeynmt.data - Loading dev data...
2021-11-27 07:16:08,704 - INFO - joeynmt.data - Loading test data...
2021-11-27 07:16:08,739 - INFO - joeynmt.data - Data loaded.
2021-11-27 07:16:08,746 - INFO - joeynmt.prediction - Process device: cuda, n_gpu: 1, batch_size per device: 3600
2021-11-27 07:16:08,746 - INFO - joeynmt.prediction - Loading model from models/enzu_transformer/latest.ckpt
2021-11-27 07:16:11,333 - INFO - joeynmt.model - Building an encoder-decoder model...
2021-11-27 07:16:11,572 - INFO - joeynmt.model - Enc-dec model built.
2021-11-27 07:16:11,651 - INFO - joeynmt.prediction - Decoding on dev set (data/enzu/dev.bpe.zu)...
2021-11-27 07:16:51,177 - INFO - joeynmt.prediction -  dev bleu[none]:   0.02 [Beam search decoding with beam size = 5 and alpha = 1.0]
2021-11-27 07:16:51,178 - INFO - joey